## Multi-Agent Orchestration: Acura MDX Winterization Workflow

This notebook implements a Retrieval-Augmented Generation (RAG) pipeline to deliver winterization recommendations for the 2022 Acura MDX. It reindexes the owner's manual using 768-dimensional embeddings and orchestrates multiple AI agents to generate actionable outputs using Google Cloud infrastructure.

### Summary of Fix

Earlier attempts failed due to dimensionality mismatches:
> Precondition failed: Query dimensionality (768) does not match database dimensionality (384)

This version resolves that by:
- Switching to the latest `text-embedding-005` model with 768-dimensional output
- Creating a new Vertex AI Matching Engine index and endpoint in `us-central1`
- Uploading correctly formatted JSON datapoints with `id`, `embedding`, and `content`

### Technologies Used

- **Vertex AI**: Hosts the vector index and serves queries via Matching Engine
- **LlamaIndex**: Handles document chunking, embedding, and interfacing with the vector store
- **Crew AI**: Provides multi-agent orchestration with role-based tasks
- **Gemini LLMs**: Used for reasoning, summarization, and checklist generation through Vertex AI

### Multi-Agent Workflow

- **Weather Agent**: Adds contextual seasonal hazards for winter
- **Manual Review Agent**: Extracts vehicle-specific winterization steps from the owner's manual
- **Recommendation Agent**: Compiles a structured winterization checklist in Markdown

This implementation serves as a reusable, extensible template for multi-agent RAG systems grounded in domain documents — integrating LLM reasoning with structured storage and enterprise-ready deployment.


```mermaid
flowchart TD
    A[User uploads PDF manual] --> B[Chunk and Embed PDF content - 768 dim]
    B --> C[Upload embeddings to Vertex AI Vector Index]
    C --> D[Create Index Endpoint]

    subgraph Vertex AI
        C
        D
    end

    E[User asks question about Acura MDX] --> F[Run query_engine query]
    F --> G[Retrieve relevant chunks]
    G --> N[Inject retrieved chunks into agent prompt]

    subgraph Crew AI Agents
        J[Weather Agent - Adds seasonal context]
        K[Manual Review Agent - Uses retrieved chunks]
        L[Recommendation Agent - Outputs checklist]
    end

    J --> M[Contextual winter risks]
    N --> K
    K --> L
    L --> I[Winterization checklist - Markdown]

    I --> H[Gemini LLM orchestrated via Crew AI]

In [ ]:
# 🧱 Cell 1: Install packages, load environment, and initialize Vertex AI

# Uncomment if packages haven't been installed yet
# !pip install --upgrade google-cloud-aiplatform python-dotenv vertexai

import os
from dotenv import load_dotenv, find_dotenv
from google.cloud import aiplatform
import vertexai

# Load .env and set ADC (Application Default Credentials)
load_dotenv(find_dotenv(), override=True)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv("GCP_KEY_PATH")

# Initialize Vertex AI
project_id = os.getenv("GCP_PROJECT_ID")
region = os.getenv("VERTEX_REGION")
vertexai.init(project=project_id, location=region)
aiplatform.init(project=project_id, location=region)

print(f"✅ Vertex AI initialized with project: {project_id}, region: {region}")

In [ ]:
# 🧱 Cell 2: Load and chunk the Acura MDX 2022 user manual PDF

import fitz  # PyMuPDF
import nltk
from pathlib import Path
from nltk.tokenize import PunktSentenceTokenizer

# Load environment variable for the PDF
pdf_local_path = os.getenv("PDF_LOCAL_PATH")
pdf_path = Path(pdf_local_path)

# Download tokenizer model
print("📦 Downloading NLTK punkt model...")
nltk.download("punkt")

# Explicitly load tokenizer
tokenizer = PunktSentenceTokenizer()

# Open the PDF
doc = fitz.open(pdf_path)
print(f"📄 Loaded PDF: {pdf_path.name} with {len(doc)} pages")

# Extract text and tokenize into sentences
all_sentences = []
for i, page in enumerate(doc):
    text = page.get_text()
    sentences = tokenizer.tokenize(text)
    all_sentences.extend(sentences)
print(f"🧠 Extracted {len(all_sentences)} sentences from all pages")

# Group into ~500-word chunks
chunks = []
chunk = ""
for sentence in all_sentences:
    if len(chunk.split()) + len(sentence.split()) <= 500:
        chunk += " " + sentence
    else:
        chunks.append(chunk.strip())
        chunk = sentence
if chunk:
    chunks.append(chunk.strip())

print(f"✅ Final chunk count: {len(chunks)} (each ≤ ~500 words)")

In [ ]:
# 🧱 Cell 3: Generate embeddings for PDF chunks using text-embedding-005

from vertexai.preview.language_models import TextEmbeddingModel
import tqdm

# Initialize the embedding model (768-dim output)
embedding_model = TextEmbeddingModel.from_pretrained("text-embedding-005")

# Embed chunks in batches to avoid memory issues
embedded_chunks = []
batch_size = 20  # tweak if needed
for i in tqdm.trange(0, len(chunks), batch_size, desc="🔢 Embedding chunks"):
    batch = chunks[i:i+batch_size]
    embeddings = embedding_model.get_embeddings(batch)
    embedded_chunks.extend([e.values for e in embeddings])

print(f"✅ Embedded {len(embedded_chunks)} chunks.")

In [ ]:
# 🧱 Cell 4 (Updated): Create new Vertex AI Tree-AH Index with 768-dim and neighbors count

from google.cloud import aiplatform

# Init Vertex AI SDK
aiplatform.init(project=os.getenv("GCP_PROJECT_ID"), location=os.getenv("VERTEX_REGION"))

new_index_display_name = "acura-mdx-index-v3-768-dim"
dimension = 768

# ✅ Create index with required approximateNeighborsCount
vs_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=new_index_display_name,
    dimensions=dimension,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
    shard_size="SHARD_SIZE_SMALL",
    index_update_method="STREAM_UPDATE",
    approximate_neighbors_count=150,  # Required for tree-AH
)

print(f"✅ Index created: {vs_index.display_name}")
print(f"🔗 Resource name: {vs_index.resource_name}")

In [ ]:
# 🧱 Cell 5: Create a new public Vertex AI Matching Engine endpoint

from google.cloud import aiplatform

# Define display name for the endpoint
endpoint_display_name = "acura-mdx-endpoint-v3-768-dim"

# Create the endpoint (public)
vs_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=endpoint_display_name,
    public_endpoint_enabled=True,
)

print(f"✅ Endpoint created: {vs_endpoint.display_name}")
print(f"🔗 Resource name: {vs_endpoint.resource_name}")

In [ ]:
# 🧱 Cell 6: Deploy index to endpoint with versioned index ID
#LONG Running process !!!

# Already available: vs_index and vs_endpoint from previous cells

deployed_index_id = "acura_mdx_v3_768_dim"

vs_deployed_index = vs_endpoint.deploy_index(
    index=vs_index,
    deployed_index_id=deployed_index_id,
    display_name=vs_index.display_name,
    machine_type="e2-standard-16",
    min_replica_count=1,
    max_replica_count=1,
)

print(f"✅ Deployed index '{vs_index.display_name}' to endpoint '{vs_endpoint.display_name}'")
print(f"🔗 Deployed Index ID: {deployed_index_id}")

In [ ]:
load_dotenv(find_dotenv(), override=True)
gcs_bucket_name_v2 = os.getenv("GCS_BUCKET_NAME_V2")
print(f"gcs_bucket_name_v2: {gcs_bucket_name_v2}")

In [ ]:
# 🧱 Cell 7A: Upload to Vertex Vector Index using LlamaIndex with explicit credentials

from google.oauth2 import service_account
from llama_index.embeddings.vertex import VertexTextEmbedding
from llama_index.vector_stores.vertexaivectorsearch import VertexAIVectorStore
from llama_index.core.schema import TextNode
from llama_index.core import StorageContext

# Load from .env
load_dotenv(find_dotenv(), override=True)
project_id = os.getenv("GCP_PROJECT_ID")
gcs_bucket_name_v2 = os.getenv("GCS_BUCKET_NAME_V2")

# Load Vertex credentials securely
key_path = os.getenv("GCP_KEY_PATH")
credentials = service_account.Credentials.from_service_account_file(key_path)

# Init embedding model
embed_model = VertexTextEmbedding(
    model_name="text-embedding-005",
    project=project_id,
    location="us-central1",
    credentials=credentials,
)

# Setup vector store
vector_store = VertexAIVectorStore(
    project_id=project_id,
    region="us-central1",
    index_id=vs_index.resource_name,
    endpoint_id=vs_endpoint.resource_name,
    gcs_bucket_name=gcs_bucket_name_v2,
)

# Convert chunks to TextNodes
nodes = []
for i, (text, embedding) in enumerate(zip(chunks, embedded_chunks)):
    node = TextNode(
        id_=f"chunk-{i}",
        text=text,
        embedding=embedding,
        metadata={"source": "Acura MDX Manual", "chunk_index": i}
    )
    nodes.append(node)

# Upload to vector store
vector_store.add(nodes)

print(f"✅ Uploaded {len(nodes)} chunks to Vertex AI Vector Store using LlamaIndex")

In [ ]:
# !pip install "google-cloud-aiplatform[preview]<1.47"

In [ ]:
# 🧱 Cell 8: Setup LlamaIndex RAG query engine (Vertex Vector Store + Gemini Pro)

from llama_index.core import VectorStoreIndex, Settings
from llama_index.llms.vertex import Vertex

# Initialize Gemini LLM for completion
llm = Vertex(
    model="gemini-pro",
    project=project_id,
    location="us-central1",
    credentials=credentials,
    temperature=0.2,
    max_tokens=1024,
)

# Assign global settings
Settings.llm = llm
Settings.embed_model = embed_model  # From previous cell

# Set up storage context from vector store
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index from storage
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    embed_model=embed_model
)

# Setup retriever-backed query engine
query_engine = index.as_query_engine()

# Ask your question!
response = query_engine.query("How do I adjust the seat in the 2022 Acura MDX?")

# Show result
print("📘 Gemini Response:")
print("-" * 80)
print(response.response)

In [55]:
from crewai import LLM
from google.oauth2 import service_account
import os

# Load credentials
gcp_key_path = os.getenv("GCP_KEY_PATH")
gcp_project = os.getenv("GCP_PROJECT_ID")

# Secure credentials object
gcp_vertex_ai_credentials = service_account.Credentials.from_service_account_file(gcp_key_path)

# ✅ Initialize Gemini LLM via Vertex AI
gemini_llm = LLM(
    model='vertex_ai/gemini-2.0-flash',
    vertex_credentials=gcp_vertex_ai_credentials,
    vertex_project=gcp_project
)

print("✅ CrewAI Gemini LLM Initialized (via Vertex)")

✅ CrewAI Gemini LLM Initialized (via Vertex)


In [56]:
from crewai import Agent

# Agent 1: Weather Context Provider
weather_agent = Agent(
    role="Weather Agent",
    goal="Add relevant seasonal context for winter preparation.",
    backstory="A weather-focused AI agent specializing in identifying the effects of cold weather on vehicles.",
    llm=gemini_llm,
    verbose=True
)

# Agent 2: Acura Manual Expert
manual_review_agent = Agent(
    role="Manual Review Agent",
    goal="Review the Acura MDX 2022 owner's manual and find relevant information about seasonal or winter use.",
    backstory="An expert in parsing owner manuals to surface relevant operational and maintenance details.",
    llm=gemini_llm,
    verbose=True
)

# Agent 3: Vehicle Winterization Advisor
recommendation_agent = Agent(
    role="Recommendation Agent",
    goal="Suggest actions for winterizing a 2022 Acura MDX using Gemini and context from the manual.",
    backstory="A certified automotive assistant trained in seasonal readiness checks, with expertise in snow, battery care, tire selection, and cold weather maintenance.",
    llm=gemini_llm,
    verbose=True
)

print("✅ Crew AI Winterization Agents Successfully Defined")

✅ Crew AI Winterization Agents Successfully Defined


In [60]:
# 🧠 Pull relevant content from vector store
query = "What winter-related features, maintenance, or recommendations are found in the 2022 Acura MDX manual?"
retrieved_chunks = query_engine.query(query)
print("✅ Retrieved chunks from query_engine.query()\n")

# 🧠 Inject retrieved text into Manual Review Agent task
from crewai import Task

# Task 1: Inject seasonal hazards
add_seasonal_context_task = Task(
    description=(
        "Describe key winter weather conditions (cold, snow, ice, low visibility) that might affect a vehicle's operation. "
        "Explain why seasonal preparation is critical. Keep it concise but informative."
    ),
    agent=weather_agent,
    expected_output="A short winter-specific context paragraph about how weather impacts vehicle operation."
)

# Task 2: Inject manual content into Manual Review Agent task
manual_insights_task = Task(
    description=(
        "Using the retrieved text from the Acura MDX manual below, identify and summarize any winter-related maintenance, safety features, "
        "or operational advice. Focus on battery care, tire pressure, fluids, heated features, VSA, and AWD.\n\n"
        f"--- BEGIN MANUAL CHUNKS ---\n{retrieved_chunks}\n--- END MANUAL CHUNKS ---"
    ),
    agent=manual_review_agent,
    expected_output="A bullet list of Acura MDX manual insights relevant to winterizing the vehicle."
)

# Task 3: Recommend a checklist using the output of the Manual Review Agent
generate_recommendations_task = Task(
    description=(
        "Using the seasonal context and the Acura MDX manual winter insights from previous tasks, generate a concise winterization checklist "
        "for a 2022 Acura MDX. Cover mechanical checks, safety checks, comfort features, and emergency prep. Output should be in clean Markdown checklist format."
    ),
    agent=recommendation_agent,
    expected_output="A Markdown-formatted winterization checklist for the Acura MDX."
)

print("✅ Tasks created using retrieved chunks for Manual Review Agent.")

✅ Retrieved chunks from query_engine.query()

✅ Tasks created using retrieved chunks for Manual Review Agent.


In [61]:
from crewai import Crew

# 🧠 Assemble the Crew using the tasks created in Cell 9
from crewai import Crew

# Create new Crew instance using tasks with injected context
winterization_crew_with_rag = Crew(
    agents=[weather_agent, manual_review_agent, recommendation_agent],
    tasks=[add_seasonal_context_task, manual_insights_task, generate_recommendations_task],
    verbose=True
)

# 🚀 Run the Crew
print("🚀 Running Crew AI with Vector-Retrieved Chunks Injected...\n")
crew_output = winterization_crew_with_rag.kickoff()

# Post-process Crew output to fix markdown rendering
clean_output = crew_output.raw.strip()
if clean_output.startswith("```markdown"):
    clean_output = clean_output.replace("```markdown", "").strip()
if clean_output.endswith("```"):
    clean_output = clean_output[:-3].strip()

from IPython.display import Markdown, display
display(Markdown(clean_output))

Overriding of current TracerProvider is not allowed


🚀 Running Crew AI with Vector-Retrieved Chunks Injected...

# Agent: Weather Agent
## Task: Describe key winter weather conditions (cold, snow, ice, low visibility) that might affect a vehicle's operation. Explain why seasonal preparation is critical. Keep it concise but informative.


# Agent: Weather Agent
## Final Answer: 
Winter weather presents numerous challenges to vehicle operation. Cold temperatures can reduce battery performance, thicken engine oil, and decrease tire pressure, leading to starting difficulties and reduced fuel efficiency. Snow and ice accumulation impair traction, increasing the risk of skidding and accidents. Low visibility due to snow, fog, or shorter daylight hours further complicates driving conditions.

Seasonal preparation is critical to mitigate these risks. Winterizing a vehicle includes checking and topping off fluids, inspecting tires, ensuring proper lighting, and equipping the vehicle with an emergency kit. These measures enhance vehicle reliabilit

## 2022 Acura MDX Winterization Checklist

**Mechanical Checks:**

*   [ ] **Winter Tires:** Install winter tires for enhanced grip in snow and ice. (Recommended)
*   [ ] **Tire Chains:** Carry tire chains if driving in areas with severe winter conditions. (Recommended)
*   [ ] **Tire Pressure:** Check and adjust tire pressure regularly. Cold weather decreases tire pressure.
*   [ ] **Wiper Blades:** Replace wiper blades to ensure effective snow and ice removal.
*   [ ] **Battery:** Have the battery tested to ensure optimal performance in cold weather.
*   [ ] **Antifreeze:** Check and top off antifreeze to prevent engine freezing.
*   [ ] **Engine Oil:** Consider using a lower viscosity oil for improved cold-weather starting.
*   [ ] **Washer Fluid:** Fill washer fluid reservoir with a winter-specific solution that prevents freezing.
*   [ ] **Brakes:** Inspect brake pads and rotors for wear and tear.

**Safety Checks:**

*   [ ] **Lights:** Ensure all headlights, taillights, brake lights, and turn signals are functioning properly.
*   [ ] **Snow Mode:** Familiarize yourself with the MDX's Snow Mode and how it adjusts vehicle settings for winter driving.
*   [ ] **All-Wheel Drive (AWD):** Understand how the AWD system functions and its benefits in winter conditions.

**Comfort Features:**

*   [ ] **Heated Seats:** Verify heated seats are functioning correctly.
*   [ ] **Heated Steering Wheel:** Check that the heated steering wheel is operating as expected.
*   [ ] **Remote Start:** Test the remote start feature to pre-warm the vehicle.
*   [ ] **Climate Control:** Ensure the heating and defrosting systems are working efficiently.

**Emergency Prep:**

*   [ ] **Emergency Kit:** Assemble a winter emergency kit including:
    *   [ ] Jumper cables
    *   [ ] Flashlight
    *   [ ] First-aid kit
    *   [ ] Blanket
    *   [ ] Warm hat and gloves
    *   [ ] Ice scraper
    *   [ ] Small shovel
    *   [ ] Sand or kitty litter for traction
    *   [ ] Snacks (non-perishable)
    *   [ ] Water
*   [ ] **Cell Phone Charger:** Keep a cell phone charger in the vehicle.
*   [ ] **Roadside Assistance:** Confirm your roadside assistance plan is active and know how to contact them.

**Driving Practices:**

*   [ ] **Drive Slowly:** Adjust your speed to the road conditions.
*   [ ] **Safe Distance:** Maintain a safe following distance.
*   [ ] **Smooth Driving:** Avoid sudden acceleration and braking.
*   [ ] **Awareness:** Be aware of your surroundings and anticipate potential hazards.

### Citation enhancement below

In [64]:
# 🧱 Cell 10: Run query to get relevant chunks (manual review input)

# You can change the query as needed
sample_query = "What winter preparation does the Acura MDX manual recommend?"

# Run query and capture both response and retrieved nodes
query_response = query_engine.query(sample_query)
retrieved_nodes = query_response.source_nodes  # These are the raw chunks with metadata

print(f"✅ Retrieved {len(retrieved_nodes)} source chunks")
print("💡 Sample chunk:\n")
print(retrieved_nodes[0].text[:500])  # Show a preview

✅ Retrieved 2 source chunks
💡 Sample chunk:

Always use the size and type of tires 
recommended in this owner’s manual. 643
uuChecking and Maintaining TiresuTire Rotation
Maintenance
Tire Rotation
Rotating tires according to the maintenance messages on the driver information 
interface helps to distribute wear more evenly and increase tire life. ■Tires without rotation marks
Rotate the tires as shown here. ■Tires with rotation marks
Rotate the tires as shown here. 1Tire Rotation
Tires with directional tread patterns should only be 
rotated


In [65]:
# 🧱 Cell 10A: Format retrieved chunks with page numbers for Crew Agent prompt

formatted_chunks = []
for node in retrieved_nodes:
    metadata = node.metadata or {}
    chunk_text = node.text.strip()
    
    # Try to get page number or fallback to chunk index
    page_info = metadata.get("page_label") or metadata.get("page_number") or f"chunk {metadata.get('chunk_index', '?')}"
    formatted = f"--- Page {page_info} ---\n{chunk_text}"
    
    formatted_chunks.append(formatted)

# Combine all for prompt use
manual_context_with_pages = "\n\n".join(formatted_chunks)

print(f"✅ Formatted {len(formatted_chunks)} chunks with page numbers.")
print("\n🔍 Sample formatted chunk:\n")
print(formatted_chunks[0][:700])  # Preview

✅ Formatted 2 chunks with page numbers.

🔍 Sample formatted chunk:

--- Page chunk 233 ---
Always use the size and type of tires 
recommended in this owner’s manual. 643
uuChecking and Maintaining TiresuTire Rotation
Maintenance
Tire Rotation
Rotating tires according to the maintenance messages on the driver information 
interface helps to distribute wear more evenly and increase tire life. ■Tires without rotation marks
Rotate the tires as shown here. ■Tires with rotation marks
Rotate the tires as shown here. 1Tire Rotation
Tires with directional tread patterns should only be 
rotated front to back (not from one side to the other). Directional tires should be mounted with the rotation 
indication mark facing forward, as shown below. Front
Rotation Mark
Front


In [66]:
# 🧱 Cell 10B: Redefine Crew Tasks with Page-Aware Context (Manual Review + Recommendation Agents)

from crewai import Task, Crew

# Manual Review Agent task with page-aware chunks
manual_insights_task_with_pages = Task(
    description=(
        "Using the retrieved and formatted text chunks below from the Acura MDX manual, "
        "identify and summarize any winter-related maintenance, safety features, or operational advice. "
        "Focus on battery care, tire pressure, fluids, heated features, VSA, AWD, and anything relevant to winterization.\n\n"
        "--- BEGIN MANUAL CONTEXT ---\n"
        f"{manual_context_with_pages}\n"
        "--- END MANUAL CONTEXT ---"
    ),
    agent=manual_review_agent,
    expected_output="A bullet list summarizing winter-related guidance and the page chunks where it was found."
)

# Recommendation Agent task with same context for citation-aligned checklist
generate_recommendations_task_with_pages = Task(
    description=(
        "Using the seasonal context and the winter-specific details from the Acura MDX manual below, "
        "generate a winterization checklist for the 2022 Acura MDX. Include mechanical checks, safety checks, comfort features, "
        "and emergency prep steps. For each checklist item, try to include the chunk/page number from which it was derived if available.\n\n"
        "--- BEGIN MANUAL CONTEXT ---\n"
        f"{manual_context_with_pages}\n"
        "--- END MANUAL CONTEXT ---"
    ),
    agent=recommendation_agent,
    expected_output="Markdown-formatted checklist with page-based citations where possible."
)

# Define Crew with updated tasks
winterization_crew_v2_citation = Crew(
    agents=[weather_agent, manual_review_agent, recommendation_agent],
    tasks=[
        add_seasonal_context_task,
        manual_insights_task_with_pages,
        generate_recommendations_task_with_pages
    ],
    verbose=True
)

# Run the crew pipeline
print("🚀 Running Crew AI (Page-Aware Winterization Workflow)...\n")
winterization_crew_v2_citation.kickoff()

Overriding of current TracerProvider is not allowed


🚀 Running Crew AI (Page-Aware Winterization Workflow)...

# Agent: Weather Agent
## Task: Describe key winter weather conditions (cold, snow, ice, low visibility) that might affect a vehicle's operation. Explain why seasonal preparation is critical. Keep it concise but informative.


# Agent: Weather Agent
## Final Answer: 
Winter weather brings a unique set of challenges to vehicle operation. **Cold temperatures** can reduce battery capacity, thicken engine oil making it harder to start, and decrease tire pressure affecting handling and braking. **Snow** reduces traction, increases stopping distances, and can lead to vehicles getting stuck. **Ice** creates extremely hazardous driving conditions due to its slipperiness, drastically reducing grip. **Low visibility** from snow, fog, or shorter daylight hours limits drivers' reaction time and increases the risk of accidents.

Seasonal preparation is critical to mitigate these risks. Winter tires provide better grip on snow and ice. Checki

CrewOutput(raw='# 2022 Acura MDX Winterization Checklist\n\n## Mechanical Checks:\n\n*   **Tire Selection (Page 644):**\n    *   If driving on snowy or frozen roads, mount all-season tires marked M+S, snow tires, or tire chains.\n    *   Select winter tires with the same size and load ranges as the original tires.\n    *   Mount winter tires on all four wheels.\n    *   If using tire chains, install them on the front tires only. Recommended chains: Konig XB-16 265 or Peerless Auto-Trac 232105.\n    *   Ensure tire chains do not touch brake lines or suspension components.\n    *   Drive slowly when using tire chains.\n    *   Be aware that summer tires are not designed for winter driving conditions. Consult your dealer for more information.\n*   **Battery (Page 645):**\n    *   Check battery terminals for corrosion monthly.\n    *   Clean corroded terminals with a baking soda and water solution. Rinse with a damp towel.\n    *   Have your vehicle inspected by a dealer if the driver info